<a href="https://colab.research.google.com/github/Tapish0305/Stock_price_prediction/blob/main/FINBERT_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING DEPENDENCIES

In [3]:
!pip install websockets>=14.0,<15.1.0


/bin/bash: line 1: 15.1.0: No such file or directory


In [4]:
!pip install yahoo_fin --no-deps


In [5]:
!pip install ta

In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import MinMaxScaler
from yahoo_fin import news
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GRU, Dense, Dropout, LSTM
from tensorflow.keras.losses import MeanSquaredError

import torch.nn.utils.rnn as rnn
import torch.nn.functional as F
import torch.nn as nn

TESLA AND APPLE STOCK PRICE NEWS DATA

In [7]:
data = pd.read_csv('/content/combined.csv')
index_values0 = data.loc[data['ticker'] == 'AAPL']
index_values1 = data.loc[data['ticker'] == 'TSLA']

In [8]:
index_values0.head()

,ticker,headline,date,eventRet,Close
22,AAPL,Despite a wave of analyst love regarding yeste...,2011-06-07,-0.032871,10.093049
23,AAPL,Bill Ackman's hedge fund made ~$490M in one da...,2011-06-14,0.002913,10.105204
28,AAPL,Apple (AAPL -1.5%) shares continue their perpl...,2011-06-20,0.001156,9.584808
47,AAPL,"After lagging for months, tech bellwethers App...",2011-07-07,0.029160,10.857841
57,AAPL,"Looking at a ""refresh of virtually every produ...",2011-07-13,0.010666,10.882765


In [9]:
index_values1.head()

,ticker,headline,date,eventRet,Close
18,TSLA,Tesla Motors (TSLA +3.6%) announces a follow-o...,2011-05-25,0.098105,1.932000
21,TSLA,Tesla Motors (TSLA +6.3%) jumps premarket afte...,2011-06-03,0.008590,2.008667
71,TSLA,Tesla Motors (TSLA +4.5%) scores a much bigger...,2011-07-20,0.053271,1.912667
103,TSLA,A video rumored to be released back-channels b...,2011-08-12,0.099013,1.754000
141,TSLA,Tesla Motors (TSLA +1.5%) gains on positive co...,2011-09-21,-0.005349,1.723333


In [10]:
news0 = index_values0[['headline','date','eventRet']]
news1 = index_values1[['headline','date','eventRet']]

In [11]:
news0['Date'] = news0['date']
news1['Date'] = news1['date']

<ipython-input-11-4546fdcd0dd7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news0['Date'] = news0['date']
<ipython-input-11-4546fdcd0dd7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news1['Date'] = news1['date']


In [12]:
news0 = news0.drop('date', axis = 1)
news1 = news1.drop('date', axis = 1)

In [13]:
news0.head()

,headline,eventRet,Date
22,Despite a wave of analyst love regarding yeste...,-0.032871,2011-06-07
23,Bill Ackman's hedge fund made ~$490M in one da...,0.002913,2011-06-14
28,Apple (AAPL -1.5%) shares continue their perpl...,0.001156,2011-06-20
47,"After lagging for months, tech bellwethers App...",0.029160,2011-07-07
57,"Looking at a ""refresh of virtually every produ...",0.010666,2011-07-13


In [14]:
news1.head()

,headline,eventRet,Date
18,Tesla Motors (TSLA +3.6%) announces a follow-o...,0.098105,2011-05-25
21,Tesla Motors (TSLA +6.3%) jumps premarket afte...,0.008590,2011-06-03
71,Tesla Motors (TSLA +4.5%) scores a much bigger...,0.053271,2011-07-20
103,A video rumored to be released back-channels b...,0.099013,2011-08-12
141,Tesla Motors (TSLA +1.5%) gains on positive co...,-0.005349,2011-09-21


TESLA AND APPLE STOCK PRICE DATA

In [15]:
# Define top companies (e.g., Apple, Microsoft, Amazon, Google, Nvidia)
top_companies0 = [ 'AAPL']
top_companies1 = ['TSLA']
# Download historical stock data
stock_data0 = yf.download(top_companies0, start="2011-05-10", end="2018-12-24", interval="1d")
stock_data1 = yf.download(top_companies1, start="2011-05-02", end="2018-12-24", interval="1d")
# Save as CSV
stock_data0.to_csv("top_sp500_stock_data.csv")
stock_data1.to_csv("top_sp500_stock_data.csv")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['TSLA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [16]:
import requests
import pandas as pd

api_key = 'ZCAD7EFH2SODPVRR.'  # paste your key here
symbol = 'AAPL'

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={api_key}&datatype=csv'
response = requests.get(url)

# Save and Load
with open(f'{symbol}.csv', 'w') as f:
    f.write(response.text)

df0 = pd.read_csv(f'{symbol}.csv')
df0.to_csv("AAPLdata.csv")


In [17]:
import requests
import pandas as pd

api_key = 'ZCAD7EFH2SODPVRR.'  # paste your key here
symbol = 'TSLA'

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={api_key}&datatype=csv'
response = requests.get(url)

# Save and Load
with open(f'{symbol}.csv', 'w') as f:
    f.write(response.text)

df1 = pd.read_csv(f'{symbol}.csv')
df1.to_csv("TSLAdata.csv")


In [18]:
stock_data0 = df0
df0.head()

,timestamp,open,high,low,close,volume
0,2025-05-09,199.00,200.5399,197.5350,198.53,36453923
1,2025-05-08,197.72,200.0500,194.6796,197.49,50478872
2,2025-05-07,199.17,199.4400,193.2500,196.25,68616943
3,2025-05-06,198.21,200.6500,197.0200,198.51,51216482
4,2025-05-05,203.10,204.1000,198.2100,198.89,69018452


In [19]:
stock_data1 = df1
df1.head()

,timestamp,open,high,low,close,volume
0,2025-05-09,290.210,307.040,290.00,298.26,132387835
1,2025-05-08,279.630,289.800,279.41,284.82,97539448
2,2025-05-07,276.880,277.920,271.00,276.22,71882408
3,2025-05-06,273.105,277.730,271.35,275.35,76715792
4,2025-05-05,284.570,284.849,274.40,280.26,94618882


In [20]:
# stock_data0 = stock_data0.droplevel(1, axis=1)
# stock_data1 = stock_data1.droplevel(1, axis=1)

In [21]:
stock_data0 = stock_data0.reset_index()
stock_data1 = stock_data1.reset_index()

In [22]:
stock_data0['Date'] = stock_data0['timestamp']
stock_data1['Date'] = stock_data1['timestamp']

In [23]:
stock_data0 = stock_data0.drop('timestamp', axis=1)
stock_data1 = stock_data1.drop('timestamp', axis =1)

In [24]:
stock_data0.head()

,index,open,high,low,close,volume,Date
0,0,199.00,200.5399,197.5350,198.53,36453923,2025-05-09
1,1,197.72,200.0500,194.6796,197.49,50478872,2025-05-08
2,2,199.17,199.4400,193.2500,196.25,68616943,2025-05-07
3,3,198.21,200.6500,197.0200,198.51,51216482,2025-05-06
4,4,203.10,204.1000,198.2100,198.89,69018452,2025-05-05


In [25]:
stock_data1.head()

,index,open,high,low,close,volume,Date
0,0,290.210,307.040,290.00,298.26,132387835,2025-05-09
1,1,279.630,289.800,279.41,284.82,97539448,2025-05-08
2,2,276.880,277.920,271.00,276.22,71882408,2025-05-07
3,3,273.105,277.730,271.35,275.35,76715792,2025-05-06
4,4,284.570,284.849,274.40,280.26,94618882,2025-05-05


CONCATENATING NEWS AND STOCK PRICE DATA ACCORDING TO DATE

In [26]:
import pandas as pd

# Ensure both DataFrames have "Date" in datetime format
stock_data0["Date"] = pd.to_datetime(stock_data0["Date"], errors="coerce")
news0["Date"] = pd.to_datetime(news0["Date"], errors="coerce")  # Fix column name if needed

# Merge the DataFrames on 'Date', keeping only common dates
merged_df0 = pd.merge(stock_data0, news0, on="Date", how="inner")

# Sort by Date and reset index
merged_df0 = merged_df0.sort_values(by="Date").reset_index(drop=True)

merged_df0.shape

(465, 9)

In [27]:
import pandas as pd

# Ensure both DataFrames have "Date" in datetime format
stock_data1["Date"] = pd.to_datetime(stock_data1["Date"], errors="coerce")
news1["Date"] = pd.to_datetime(news1["Date"], errors="coerce")  # Fix column name if needed

# Merge the DataFrames on 'Date', keeping only common dates
merged_df1 = pd.merge(stock_data1, news1, on="Date", how="inner")

# Sort by Date and reset index
merged_df1 = merged_df1.sort_values(by="Date").reset_index(drop=True)

merged_df1.shape

(483, 9)

In [28]:
merged_df0.dropna(inplace = True)
merged_df1.dropna(inplace = True)

In [29]:
merged_df0.shape

(463, 9)

In [30]:
merged_df1.shape

(481, 9)

ADDING NEW FEATURE

In [31]:
merged_df0['Daily_Return'] = stock_data0['close'].pct_change()
merged_df1['Daily_Return'] = stock_data1['close'].pct_change()

In [32]:
## EXPONENTIAL MOVING AVERAGE
merged_df0['EMA_100'] = merged_df0['close'].ewm(span=10, adjust=False).mean()
merged_df1['EMA_100'] = merged_df1['close'].ewm(span=10, adjust=False).mean()
## BOLLINGER DEMANDS
import ta
merged_df0['RSI'] = ta.momentum.RSIIndicator(merged_df0['close'], window=14).rsi()
merged_df1['RSI'] = ta.momentum.RSIIndicator(merged_df1['close'], window=14).rsi()
## Volatility
merged_df0['Volatility_10'] = merged_df0['Daily_Return'].rolling(window=10).std()
merged_df1['Volatility_10'] = merged_df1['Daily_Return'].rolling(window=10).std()

In [33]:
price0 = merged_df0.dropna()
price1 = merged_df1.dropna()

In [34]:
price0.head()

,index,open,high,low,close,volume,Date,headline,eventRet,Daily_Return,EMA_100,RSI,Volatility_10
14,3446,365.08,375.45,365.0000,373.72,31119500,2011-08-25,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,0.026750,-0.032943,371.243555,63.015414,0.019059
15,3446,365.08,375.45,365.0000,373.72,31119500,2011-08-25,"Let's get real: In the long run, Apple (AAPL -...",0.026750,0.019776,371.693818,63.015414,0.020630
16,3445,371.17,383.80,370.8008,383.58,22909900,2011-08-26,Apple (AAPL +2.6%) shares rallied today to ful...,0.036503,-0.013758,373.854942,68.061729,0.014221
17,3430,397.00,413.23,395.2045,411.63,29423600,2011-09-19,Apple (AAPL +2%) is outperforming strongly tod...,0.051400,0.040511,380.723134,77.476811,0.021259
18,3425,399.86,403.98,391.3000,403.17,29031300,2011-09-26,"U.S. shares are higher across the board, with ...",-0.006321,0.001880,384.804382,70.706704,0.021354


In [35]:
price1.head()

,index,open,high,low,close,volume,Date,headline,eventRet,Daily_Return,EMA_100,RSI,Volatility_10
13,3302,34.97,35.1500,34.3000,34.40,522400,2012-03-22,Luxury electric auto maker Tesla Motors (TSLA ...,-0.025205,-0.050929,29.037836,62.384762,0.037984
14,3300,35.59,38.0900,35.0414,37.40,3140500,2012-03-26,Tesla Motors (TSLA +3.7%) is upgraded to Buy f...,0.102554,-0.043997,30.558230,66.732870,0.037286
15,3294,36.70,38.4696,36.6700,38.01,1098100,2012-04-03,Shares of Tesla Motors (TSLA +2.9%) kick up a ...,-0.057820,0.060967,31.913097,67.554146,0.043372
16,3291,34.10,34.2900,33.1000,33.15,1655700,2012-04-09,Abu Dhabi National Energy sells off its stake ...,-0.074245,0.000746,32.137989,55.746072,0.043612
17,3269,30.30,30.7700,29.7600,30.06,1947900,2012-05-09,More on Tesla Motors (NASDAQ:TSLA): Q1 misses ...,0.021949,0.051998,31.760172,49.787353,0.048149


In [36]:
price0['volume'] = price0['volume']/10e6
price1['volume'] = price1['volume']/10e6

<ipython-input-36-0b75dc969087>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price0['volume'] = price0['volume']/10e6
<ipython-input-36-0b75dc969087>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price1['volume'] = price1['volume']/10e6


Performing Roubust Scaling on data

In [37]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cs0 = scaler.fit_transform( price0[['close','high','low','open','volume','eventRet',
                                                                              'Daily_Return','EMA_100','RSI','Volatility_10']])
df0 = pd.DataFrame(cs0)

In [38]:
cs1 = scaler.fit_transform( price1[['close','high','low','open','volume','eventRet',
                                                                              'Daily_Return','EMA_100','RSI','Volatility_10']])
df1 = pd.DataFrame(cs1)

Assigning Features Name

In [39]:
df0.rename(columns={0:'close',1:'high',2:'low',3:'open',
                   4: 'volume',5:'eventRet',6:'Daily_Return',
                   7:'EMA_100', 8:'RSI', 9: 'Volatility_10'}, inplace=True)

In [40]:
df1.rename(columns={0:'close',1:'high',2:'low',3:'open',
                   4: 'volume',5:'eventRet',6:'Daily_Return',
                   7:'EMA_100', 8:'RSI', 9: 'Volatility_10'}, inplace=True)

In [41]:
price0 = price0.reset_index()
price1 = price1.reset_index()

In [42]:
df0['headline']	= price0['headline']
df0['date'] = price0['Date']

In [43]:
df1['headline']	= price1['headline']
df1['date'] = price1['Date']

In [44]:
df0.head()

,close,high,low,open,volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,0.686543,0.681157,0.667464,0.660474,-0.044611,0.498949,-1.975327,0.703693,0.353346,0.835047,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,2011-08-25
1,0.686543,0.681157,0.667464,0.660474,-0.044611,0.498949,1.324279,0.705335,0.353346,1.049078,"Let's get real: In the long run, Apple (AAPL -...",2011-08-25
2,0.721047,0.710185,0.688009,0.682003,-0.423843,0.684177,-0.774564,0.713215,0.543421,0.175599,Apple (AAPL +2.6%) shares rallied today to ful...,2011-08-26
3,0.819205,0.812494,0.774444,0.773317,-0.122951,0.967096,2.622004,0.738260,0.898050,1.134913,Apple (AAPL +2%) is outperforming strongly tod...,2011-09-19
4,0.789600,0.780338,0.760615,0.783427,-0.141073,-0.129124,0.204169,0.753142,0.643047,1.147859,"U.S. shares are higher across the board, with ...",2011-09-26


In [45]:
df1.head()

,close,high,low,open,volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-1.563315,-1.630168,-1.566213,-1.574579,-1.037450,-0.237777,-1.217214,-1.849000,0.385113,0.228178,Luxury electric auto maker Tesla Motors (TSLA ...,2012-03-22
1,-1.537921,-1.604645,-1.559731,-1.569238,-0.640553,1.150989,-1.052500,-1.834131,0.596026,0.187868,Tesla Motors (TSLA +3.7%) is upgraded to Buy f...,2012-03-26
2,-1.532758,-1.601349,-1.545490,-1.559676,-0.950175,-0.592316,1.441556,-1.820881,0.635863,0.539320,Shares of Tesla Motors (TSLA +2.9%) kick up a ...,2012-04-03
3,-1.573895,-1.637634,-1.576706,-1.582073,-0.865645,-0.770850,0.010637,-1.818682,0.063091,0.553172,Abu Dhabi National Energy sells off its stake ...,2012-04-09
4,-1.600051,-1.668193,-1.605911,-1.614808,-0.821348,0.274792,1.228431,-1.822377,-0.225948,0.815173,More on Tesla Motors (NASDAQ:TSLA): Q1 misses ...,2012-05-09


# Using Finbert to performing sentiment analysis on stock price news data

In [46]:
!pip install feedparser

In [47]:
import feedparser
import pandas as pd

In [48]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load FinBERT model (trained for financial sentiment analysis)
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [49]:
df0.head()

,close,high,low,open,volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,0.686543,0.681157,0.667464,0.660474,-0.044611,0.498949,-1.975327,0.703693,0.353346,0.835047,Apple (NASDAQ:AAPL) is now down just 1.6% in t...,2011-08-25
1,0.686543,0.681157,0.667464,0.660474,-0.044611,0.498949,1.324279,0.705335,0.353346,1.049078,"Let's get real: In the long run, Apple (AAPL -...",2011-08-25
2,0.721047,0.710185,0.688009,0.682003,-0.423843,0.684177,-0.774564,0.713215,0.543421,0.175599,Apple (AAPL +2.6%) shares rallied today to ful...,2011-08-26
3,0.819205,0.812494,0.774444,0.773317,-0.122951,0.967096,2.622004,0.738260,0.898050,1.134913,Apple (AAPL +2%) is outperforming strongly tod...,2011-09-19
4,0.789600,0.780338,0.760615,0.783427,-0.141073,-0.129124,0.204169,0.753142,0.643047,1.147859,"U.S. shares are higher across the board, with ...",2011-09-26


In [50]:
df1.head()

,close,high,low,open,volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,headline,date
0,-1.563315,-1.630168,-1.566213,-1.574579,-1.037450,-0.237777,-1.217214,-1.849000,0.385113,0.228178,Luxury electric auto maker Tesla Motors (TSLA ...,2012-03-22
1,-1.537921,-1.604645,-1.559731,-1.569238,-0.640553,1.150989,-1.052500,-1.834131,0.596026,0.187868,Tesla Motors (TSLA +3.7%) is upgraded to Buy f...,2012-03-26
2,-1.532758,-1.601349,-1.545490,-1.559676,-0.950175,-0.592316,1.441556,-1.820881,0.635863,0.539320,Shares of Tesla Motors (TSLA +2.9%) kick up a ...,2012-04-03
3,-1.573895,-1.637634,-1.576706,-1.582073,-0.865645,-0.770850,0.010637,-1.818682,0.063091,0.553172,Abu Dhabi National Energy sells off its stake ...,2012-04-09
4,-1.600051,-1.668193,-1.605911,-1.614808,-0.821348,0.274792,1.228431,-1.822377,-0.225948,0.815173,More on Tesla Motors (NASDAQ:TSLA): Q1 misses ...,2012-05-09


In [51]:
def get_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get label with highest probability
    probs = F.softmax(outputs.logits, dim=-1)

    # Convert label to sentiment
    return (probs[0][2]-probs[0][0]).numpy().item() ## positive - negative

In [52]:
df0.columns

Index(['close', 'high', 'low', 'open', 'volume', 'eventRet', 'Daily_Return',
       'EMA_100', 'RSI', 'Volatility_10', 'headline', 'date'],
      dtype='object')

In [53]:
df0['sentiment'] = df0['headline'].apply(get_sentiment)
df1['sentiment'] = df1['headline'].apply(get_sentiment)

In [54]:
df0 = df0.drop(['headline','date'], axis=1)
df1 = df1.drop(['headline','date'], axis=1)

In [55]:
df0.head()

,close,high,low,open,volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,sentiment
0,0.686543,0.681157,0.667464,0.660474,-0.044611,0.498949,-1.975327,0.703693,0.353346,0.835047,0.008747
1,0.686543,0.681157,0.667464,0.660474,-0.044611,0.498949,1.324279,0.705335,0.353346,1.049078,0.415604
2,0.721047,0.710185,0.688009,0.682003,-0.423843,0.684177,-0.774564,0.713215,0.543421,0.175599,-0.926992
3,0.819205,0.812494,0.774444,0.773317,-0.122951,0.967096,2.622004,0.738260,0.898050,1.134913,-0.538386
4,0.789600,0.780338,0.760615,0.783427,-0.141073,-0.129124,0.204169,0.753142,0.643047,1.147859,-0.920180


In [56]:
df1.head()

,close,high,low,open,volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,sentiment
0,-1.563315,-1.630168,-1.566213,-1.574579,-1.037450,-0.237777,-1.217214,-1.849000,0.385113,0.228178,0.015583
1,-1.537921,-1.604645,-1.559731,-1.569238,-0.640553,1.150989,-1.052500,-1.834131,0.596026,0.187868,-0.842692
2,-1.532758,-1.601349,-1.545490,-1.559676,-0.950175,-0.592316,1.441556,-1.820881,0.635863,0.539320,-0.918571
3,-1.573895,-1.637634,-1.576706,-1.582073,-0.865645,-0.770850,0.010637,-1.818682,0.063091,0.553172,0.536333
4,-1.600051,-1.668193,-1.605911,-1.614808,-0.821348,0.274792,1.228431,-1.822377,-0.225948,0.815173,-0.915175


In [57]:
df0.shape

(450, 11)

In [58]:
df1.shape

(468, 11)

In [59]:
df1.head()

,close,high,low,open,volume,eventRet,Daily_Return,EMA_100,RSI,Volatility_10,sentiment
0,-1.563315,-1.630168,-1.566213,-1.574579,-1.037450,-0.237777,-1.217214,-1.849000,0.385113,0.228178,0.015583
1,-1.537921,-1.604645,-1.559731,-1.569238,-0.640553,1.150989,-1.052500,-1.834131,0.596026,0.187868,-0.842692
2,-1.532758,-1.601349,-1.545490,-1.559676,-0.950175,-0.592316,1.441556,-1.820881,0.635863,0.539320,-0.918571
3,-1.573895,-1.637634,-1.576706,-1.582073,-0.865645,-0.770850,0.010637,-1.818682,0.063091,0.553172,0.536333
4,-1.600051,-1.668193,-1.605911,-1.614808,-0.821348,0.274792,1.228431,-1.822377,-0.225948,0.815173,-0.915175


In [139]:
X_train0 = df0[50:]
X_valid0 = df0[:50]

Creating time series sequence as input to GRU and LSTM

In [140]:
def create_seq(df, seq_len):
  x_train = []
  y_train = []
  for i in range(len(df)-seq_len):
    x_train.append(df.iloc[i:(i+seq_len)])
    y_train.append(df.iloc[i+seq_len,0])
  return np.array(x_train), np.array(y_train)

In [141]:
x_train0, y_train0 = create_seq(X_train0, seq_len = 45)
x_valid0, y_valid0 = create_seq(X_valid0, seq_len = 45) ## made from df0

In [142]:
len(x_valid0[0])

45

In [143]:
x1, y1 = create_seq(df1, seq_len = 50) ## made from df1

# GRU MODEL

In [163]:
model= tf.keras.Sequential([
    GRU(32, return_sequences=True, kernel_regularizer=l2(0.01)),  # L2 Regularization
    Dropout(0.2),  # 20% neurons ko random drop
    Dense(1)  # Final output layer
])
model.compile(loss='mean_squared_error',optimizer='adam')

In [164]:
model.fit(x_train0, y_train0, epochs = 25, validation_data= (x_valid0, y_valid0))

Epoch 1/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - loss: 0.5853 - val_loss: 1.2014
Epoch 2/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.4670 - val_loss: 1.0258
Epoch 3/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.4159 - val_loss: 1.1557
Epoch 4/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.4075 - val_loss: 1.0975
Epoch 5/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.3896 - val_loss: 1.0319
Epoch 6/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.3764 - val_loss: 1.0542
Epoch 7/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.3735 - val_loss: 1.0860
Epoch 8/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.3276 - val_loss: 1.0798
Epoch 9/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.3639 - val_loss: 0.9677
Epoch 10/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.3160 - val_loss: 0.9988
Epoch 11/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.3120 - val_loss: 0.9849
Epoch 12/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.3

In [165]:
y_pred1 = model.predict(x1)

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


In [166]:
mse = tf.keras.losses.MeanSquaredError()

In [167]:
mean_absolute_error0 = mse(y_pred1, y1)
# mean_absolute_error1 = mse(y_pred1, y1)

In [168]:
mean_absolute_error0

<tf.Tensor: shape=(), dtype=float32, numpy=0.39321041107177734>

In [189]:
model.save('GRU_model.h5')

In [192]:
import pickle

# Convert model architecture to JSON
model_json = model.to_json()

# Save JSON to .pkl file
with open("model_architecture.pkl", "wb") as json_file:
    pickle.dump(model_json, json_file)


# LSTM MODEL

In [169]:
model1= tf.keras.Sequential([
    LSTM(32, return_sequences=True, kernel_regularizer=l2(0.001)),  # L2 Regularization
    Dropout(0.2),
    Dense(1),
])
model1.compile(loss='mean_squared_error',optimizer='adam')

In [170]:
model1.fit(x_train0, y_train0, epochs = 25, validation_data= (x_valid0, y_valid0))

Epoch 1/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.2640 - val_loss: 0.8082
Epoch 2/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2796 - val_loss: 0.9048
Epoch 3/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.2626 - val_loss: 0.9645
Epoch 4/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2868 - val_loss: 0.8396
Epoch 5/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.2690 - val_loss: 0.9456
Epoch 6/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.2905 - val_loss: 0.9002
Epoch 7/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.2669 - val_loss: 0.8485
Epoch 8/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.2708 - val_loss: 0.8517
Epoch 9/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.2676 - val_loss: 0.8084
Epoch 10/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.2423 - val_loss: 0.8770
Epoch 11/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.2405 - val_loss: 0.8164
Epoch 12/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.2

In [171]:
y_pred1 = model1.predict(x1)

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


In [172]:
mean_absolute_error1 = mse(y_pred1, y1)

In [174]:
mean_absolute_error1

<tf.Tensor: shape=(), dtype=float32, numpy=0.3954583704471588>

In [190]:
model1.save('LSTM.h5')

In [193]:
import pickle

# Convert model architecture to JSON
model1_json = model1.to_json()

# Save JSON to .pkl file
with open("model1_architecture.pkl", "wb") as json_file:
    pickle.dump(model1_json, json_file)


# ADVERSINAL TRAINING

In [175]:
model2 = tf.keras.Sequential([
      GRU(32, return_sequences=True, kernel_regularizer=l2(0.001)),  # L2 Regularization
      Dropout(0.3),  # 30% neurons ko random drop karenge
      GRU(16, return_sequences=False, kernel_regularizer=l2(0.001)),
      Dropout(0.3),
      Dense(1)  # Final output layer
  ])

In [176]:
model2.compile(loss='mean_squared_error',optimizer='adam')

In [177]:
mse = MeanSquaredError()
def create_adversarial_pattern(model, x, y_true, epsilon=0.01, alpha=0.001, num_steps=7):
     x = tf.convert_to_tensor(x, dtype=tf.float32)
     y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
      # Initialize adversarial example with the original input
     adversarial_x = tf.identity(x)
     for _ in range(num_steps):
         with tf.GradientTape() as tape:
             tape.watch(adversarial_x)
             y_pred = model(adversarial_x, training=True)
             loss = mse(y_true, y_pred)

         # Compute the gradient
         gradient = tape.gradient(loss, adversarial_x)
         signed_grad = tf.sign(gradient)  # Get sign of the gradient
         # Apply small perturbation step
         adversarial_x = adversarial_x + alpha * signed_grad
         # Project the adversarial example to ensure it stays in the epsilon-ball
         perturbation = tf.clip_by_value(adversarial_x - x, -epsilon, epsilon)
         adversarial_x = tf.clip_by_value(x + perturbation, 0, 1)  # Ensure valid input range

     return adversarial_x.numpy()

In [178]:
x_train_adv = create_adversarial_pattern(model2, x_train0, y_train0)

In [179]:
x_valid_adv = create_adversarial_pattern(model2, x_valid0, y_valid0)

In [180]:
x_train_adv.shape

(355, 45, 11)

In [181]:
x_train_combined = np.vstack([x_train0, x_train_adv])
y_train_combined = np.hstack([y_train0, y_train0])

In [182]:
x_valid_combined = np.vstack([x_valid0, x_valid_adv])
y_valid_combined = np.hstack([y_valid0, y_valid0])

In [183]:
model2.fit(x_train_combined, y_train_combined, validation_data =(x_valid_combined, y_valid_combined),  epochs = 17)

Epoch 1/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 114ms/step - loss: 0.2822 - val_loss: 0.0841
Epoch 2/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - loss: 0.1289 - val_loss: 0.1325
Epoch 3/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 103ms/step - loss: 0.1068 - val_loss: 0.0906
Epoch 4/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 131ms/step - loss: 0.0978 - val_loss: 0.0816
Epoch 5/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 115ms/step - loss: 0.0884 - val_loss: 0.0984
Epoch 6/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 102ms/step - loss: 0.0807 - val_loss: 0.0905
Epoch 7/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 0.0867 - val_loss: 0.0490
Epoch 8/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - loss: 0.0749 - val_loss: 0.0603
Epoch 9/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - loss: 0.0750 - val_loss: 0.0400
Epoch 10/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0638 - val_loss: 0.0449
Epoch 11/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0647 - val_loss: 0.0583
Epoch 12/17
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - lo

In [184]:
y_pred_adv0 = model2.predict(x_valid0)
y_pred1= model2.predict(x1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 652ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [185]:
mean_absolute_error0 = mse(y_pred_adv0, y_valid0)
mean_absolute_error1= mse(y_pred1, y1)

In [186]:
mean_absolute_error0

<tf.Tensor: shape=(), dtype=float32, numpy=0.016835249960422516>

In [187]:
mean_absolute_error1

<tf.Tensor: shape=(), dtype=float32, numpy=0.024143563583493233>

In [191]:
model2.save("Adversinal.h5")

In [194]:
import pickle

# Convert model architecture to JSON
model2_json = model2.to_json()

# Save JSON to .pkl file
with open("model2_architecture.pkl", "wb") as json_file:
    pickle.dump(model2_json, json_file)
